<a href="https://colab.research.google.com/github/rai02/Major_Project/blob/main/test9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
import pandas as pd
import numpy as np

In [22]:
cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv("/content/drive/MyDrive/dwdmData/ratings.csv")

In [23]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [24]:
ratings = ratings.drop(['timestamp'],axis=1)

In [25]:
ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [37]:
movieProperties = ratings.groupby('movieId').agg({'rating': [np.size, np.mean]})
movieProperties.head(1076)

rating          
          size      mean
movieId                 
1        215.0  3.920930
2        110.0  3.431818
3         52.0  3.259615
4          7.0  2.357143
5         49.0  3.071429
...        ...       ...
1392       4.0  4.125000
1393      85.0  3.594118
1394      58.0  3.991379
1395       5.0  3.000000
1396      46.0  3.478261

[1076 rows x 2 columns]

In [27]:
movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
movieNormalizedNumRatings.head()

,size
movieId,
1,0.652439
2,0.332317
3,0.155488
4,0.018293
5,0.146341


In [28]:
movies = pd.read_csv("/content/drive/MyDrive/dwdmData/movies.csv")

In [29]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [30]:
def fun(g):
    l = [   "Action",
            "Adventure",
            "Animation",
            "Children",
            "Comedy",
            "Crime",
            "Documentary",
            "Drama",
            "Fantasy",
            "Film-Noir",
            "Horror",
            "Musical",
            "Mystery",
            "Romance",
            "Sci-Fi",
            "Thriller",
            "War",
            "Western",
            "(no genres listed)"]
    l1 = []
    for genre in l:
        if(genre in g):
            # print(genre)
            l1.append(1)
        else :
            l1.append(0)
    # print(l1)
    return l1


In [39]:
movie_dict = {}
for index, row in movies.iterrows():
    genres = row['genres'].split('|')
    movie_id = row['movieId']
    name = row['title']
    genres_1 = map(str, genres)

    # print(movie_id,name,list(genres_1),movieNormalizedNumRatings.loc[movie_id].get('size'), movieProperties.loc[movie_id].rating.get('mean'))
    try:
        d = {
          'name': name,
          'genres': fun(genres),
          'size': movieNormalizedNumRatings.loc[movie_id].get('size'),
          'mean_rating': movieProperties.loc[movie_id].rating.get('mean')
      }
        movie_dict[movie_id] = d
    except:
      print("skip")
    
    
    # # movie_dict[movie_id] = (name,genres,movieNormalizedNumRatings.loc[movie_id].get('size'), movieProperties.loc[movie_id].rating.get('mean'))
    # # break
    # # print(row['title'], row['genres'])

skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip


In [40]:
movie_dict[1]

{'genres': [0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'mean_rating': 3.9209302325581397,
 'name': 'Toy Story (1995)',
 'size': 0.6524390243902439}

In [41]:
from scipy import spatial

def ComputeDistance(a, b):
    genresA = a['genres']
    genresB = b['genres']
    genreDistance = spatial.distance.cosine(genresA, genresB)
    popularityA = a['size']
    popularityB = b['size']
    popularityDistance = abs(popularityA - popularityB)
    return genreDistance + popularityDistance

ComputeDistance(movie_dict[1], movie_dict[4])


1.3759474517162535

In [49]:
import operator

def getNeighbors(movieID, K):
    distances = []
    for movie in movie_dict:
        if (movie != movieID):
            dist = ComputeDistance(movie_dict[movieID], movie_dict[movie])
            distances.append((movie, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

K = 10
avgRating = 0

# print(movie_dict[1], '***\n')
neighbors = getNeighbors(1, K) # Toy Story (1995)
print(f"Most Similar movies to {movie_dict[1]['name']} are :")
# print(neighbors)
for neighbor in neighbors:
    avgRating += movie_dict[neighbor]['mean_rating']
    print (movie_dict[neighbor]['name'] + " " + str(movie_dict[neighbor]['mean_rating']))

avgRating /= K
print("**************************************************************************")
print(f"Predicted average raiting : {avgRating}")


Most Similar movies to Toy Story (1995) are :
Shrek (2001) 3.8676470588235294
Monsters, Inc. (2001) 3.871212121212121
Aladdin (1992) 3.7923497267759565
Finding Nemo (2003) 3.9609929078014185
Toy Story 2 (1999) 3.8608247422680413
Lord of the Rings: The Fellowship of the Ring, The (2001) 4.106060606060606
Lord of the Rings: The Two Towers, The (2002) 4.0212765957446805
Monty Python and the Holy Grail (1975) 4.161764705882353
Incredibles, The (2004) 3.836
Bug's Life, A (1998) 3.516304347826087
**************************************************************************
Predicted average raiting : 3.899443281239479
